**Install the Kaggle package.**

In [1]:
!pip install -q kaggle

Get Your Kaggle API Token:

Go to your Kaggle account.

**Click on your profile picture on the top right, and go to Account**
*   Scroll down to the API section and click on Create New API Token.
*   This will download a kaggle.json file containing your API credentials.

Upload this

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jibinn","key":"dcfb78ea9d8c716b6d08aeb4110600e5"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d gpiosenka/100-bird-species


100% 1.95G/1.96G [00:51<00:00, 40.1MB/s]
100% 1.96G/1.96G [00:51<00:00, 40.9MB/s]


In [11]:
!unzip 100-bird-species.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/WHITE TAILED TROPIC/120.jpg  
  inflating: train/WHITE TAILED TROPIC/121.jpg  
  inflating: train/WHITE TAILED TROPIC/122.jpg  
  inflating: train/WHITE TAILED TROPIC/123.jpg  
  inflating: train/WHITE TAILED TROPIC/124.jpg  
  inflating: train/WHITE TAILED TROPIC/125.jpg  
  inflating: train/WHITE TAILED TROPIC/126.jpg  
  inflating: train/WHITE TAILED TROPIC/127.jpg  
  inflating: train/WHITE TAILED TROPIC/128.jpg  
  inflating: train/WHITE TAILED TROPIC/129.jpg  
  inflating: train/WHITE TAILED TROPIC/130.jpg  
  inflating: train/WHITE TAILED TROPIC/131.jpg  
  inflating: train/WHITE TAILED TROPIC/132.jpg  
  inflating: train/WHITE TAILED TROPIC/133.jpg  
  inflating: train/WHITE TAILED TROPIC/134.jpg  
  inflating: train/WHITE TAILED TROPIC/135.jpg  
  inflating: train/WHITE TAILED TROPIC/136.jpg  
  inflating: train/WHITE TAILED TROPIC/137.jpg  
  inflating: train/WHITE TAILED TROPIC/138.jpg  
  inflating: train

In [12]:
!pip install -U tensorflow
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [17]:
!ls

 100-bird-species.zip  'EfficientNetB0-525-(224 X 224)- 98.97.h5'   sample_data   train
 birds.csv	        kaggle.json				    test	  valid


In [18]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'train',  # Updated path for training data
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'  # Changed to 'categorical' as the dataset likely has multiple bird species
)

validation_generator = validation_datagen.flow_from_directory(
    'valid',  # Updated path for validation data
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'  # Changed to 'categorical'
)


Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


In [19]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)


Epoch 1/10
2645/2645 [==============================] - 554s 209ms/step - loss: 0.0145 - accuracy: 0.9977 - val_loss: 0.0138 - val_accuracy: 0.9981
Epoch 2/10
2645/2645 [==============================] - 494s 187ms/step - loss: 0.0138 - accuracy: 0.9981 - val_loss: 0.0138 - val_accuracy: 0.9981
Epoch 3/10
2645/2645 [==============================] - 468s 177ms/step - loss: 0.0138 - accuracy: 0.9981 - val_loss: 0.0138 - val_accuracy: 0.9981
Epoch 4/10
2645/2645 [==============================] - 468s 177ms/step - loss: 0.0138 - accuracy: 0.9981 - val_loss: 0.0138 - val_accuracy: 0.9981
Epoch 5/10
2645/2645 [==============================] - 471s 178ms/step - loss: 0.0138 - accuracy: 0.9981 - val_loss: 0.0138 - val_accuracy: 0.9981
Epoch 6/10
2645/2645 [==============================] - 488s 185ms/step - loss: 0.0138 - accuracy: 0.9981 - val_loss: 0.0138 - val_accuracy: 0.9981
Epoch 7/10
2645/2645 [==============================] - 474s 179ms/step - loss: 0.0138 - accuracy: 0.9981 - val_

In [20]:
from google.colab import files

uploaded = files.upload()


Saving bird1.jpg to bird1.jpg


In [23]:
!ls


 100-bird-species.zip   birds.csv				    kaggle.json   test	  valid
 bird1.jpg	       'EfficientNetB0-525-(224 X 224)- 98.97.h5'   sample_data   train


In [24]:
from tensorflow.keras.preprocessing import image

def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(64, 64))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.  # Normalize the image
    return img_tensor

# Load and preprocess the image
test_image = preprocess_image('bird1.jpg')

# Predict using the model
prediction = model.predict(test_image)

# Check if the top predicted class is "bird"
predicted_class = np.argmax(prediction, axis=1)
bird_index = 0  # Replace with the index of "bird" in your dataset

if predicted_class[0] == bird_index:
    print("The image is a bird.")
else:
    print("The image is not a bird.")


1/1 [==============================] - 0s 32ms/step
The image is a bird.
